**EVENT DETECTION**


The script provides a comprehensive implementation I-DT (Identification by Dispersion Threshold) algorithms, as described in the paper.

---
The script analyzes eye-tracking data to detect and classify eye movements into fixations and saccades.


*   I-DT (Identification by Dispersion Threshold): This method uses the dispersion of gaze points within a time window to classify movements. If the points remain within a certain spatial threshold, it indicates a fixation.







In [ ]:
#Installing required libraries
!pip install pandas

In [ ]:
#Import required libraries
import pandas as pd
import numpy as np

Section 1: Data Loading
-----------------------
This section reads a CSV file into a pandas DataFrame. This file contains eye-tracking data including timestamps, gaze points, participant information, validity of the data points and other features.


In [ ]:
# Read the CSV file into a DataFrame
df = pd.read_csv('ExampleData.csv')

Section 2: Data Filtering
-------------------------
* Filters the DataFrame to only include data for 'Participant1'. This is useful in scenarios where the dataset contains multiple participants, and analysis needs to be participant-specific.

* Further filters the data to include only rows where both the 'Validity left' and 'Validity right' fields are 'Valid'.

* This step ensures that only high-quality, valid gaze data is considered in the analysis.

In [ ]:
# Filter the DataFrame for rows where 'Participant name' is 'Participant1'
df = df[df['Participant name'] == 'Participant1']

# Further filter the DataFrame for rows where both 'Validity left' and 'Validity right' are 'Valid'
df = df[(df['Validity left'] == 'Valid') & (df['Validity right'] == 'Valid')]
df = df.reset_index(drop=True)



Section 3: Function Definitions - Identification by Dispersion Threshold (I-DT)
-------------------------------
* This section contains definitions of several functions used for processing eye-tracking data.

In [ ]:
# Define the I-DT algorithm for classifying eye movements based on dispersion within a time window.
def idt_algorithm(data, d_threshold, t_threshold, epsilon=0):
    # Initialize arrays for storing eye movement IDs and types.
    eye_movement_ids = [0] * len(data)  # 0 implies non-fixation/saccade by default.
    eye_movement_types = ['saccade'] * len(data)  # Default to 'saccade'.
    current_id = 1  # Start ID for fixations.
    i = 0
    n = len(data)
    centroids = []  # List to store fixation centroids.

    # Process each point in the DataFrame based on time windows.
    while i < n:
        start_time = data.iloc[i]['Computer timestamp']
        window_data = filter_points_using_window(data, start_time, t_threshold + epsilon)
        # Calculate dispersion in the window.
        disp_x = window_data['Gaze point X'].max() - window_data['Gaze point X'].min()
        disp_y = window_data['Gaze point Y'].max() - window_data['Gaze point Y'].min()
        dispersion = disp_x + disp_y

        # Enlarge the window until the dispersion exceeds the threshold or no more data can be added.
        while dispersion <= d_threshold:
            old_window_data = window_data
            window_data = enlarge_window(data, window_data)
            if window_data.empty or len(window_data) == len(data) or len(old_window_data) == len(window_data):
                break
            disp_x = window_data['Gaze point X'].max() - window_data['Gaze point X'].min()
            disp_y = window_data['Gaze point Y'].max() - window_data['Gaze point Y'].min()
            dispersion = disp_x + disp_y

        # Classify the window as a fixation if dispersion criteria are finally exceeded.
        if dispersion > d_threshold and len(window_data) > 10:  # Check window size to ensure enough data points.
            for index in window_data.index:
                eye_movement_ids[index] = current_id
                eye_movement_types[index] = 'fixation'
            centroid_x, centroid_y = calculate_fixation_centroid(window_data)
            centroids.append((current_id, centroid_x, centroid_y))
            current_id += 1

        i += len(window_data)

    return eye_movement_types, eye_movement_ids, centroids

# Helper functions used by the I-DT algorithm
def filter_points_using_window(data, start_time, time_threshold):
    """Filter data within a specified time window."""
    return data[(data['Computer timestamp'] >= start_time) & (data['Computer timestamp'] <= start_time + time_threshold)]

def enlarge_window(data, current_data):
    """Expand the current data window to include the next point."""
    if current_data.index[-1] + 1 < len(data):
        return pd.concat([current_data, data.iloc[current_data.index[-1] + 1:current_data.index[-1] + 2]])
    return current_data

def calculate_fixation_centroid(fixation_data):
    """Calculate the geometric center of fixation points."""
    centroid_x = fixation_data['Gaze point X'].mean()
    centroid_y = fixation_data['Gaze point Y'].mean()
    return (centroid_x, centroid_y)

Section 4: Data Processing - Eye Movement Event Detection

---------------------------
This section applies the I-DT algorithm to classify eye movements, and calculates the centroids of each fixation.

In [ ]:
# Parameters for the I-DT algorithm
D_THRESHOLD = 150  # Maximum allowed dispersion to classify as fixation.
T_THRESHOLD = 200  # Minimum duration in milliseconds for potential fixations.

# Execute the I-DT algorithm and update the DataFrame with classifications and centroids.
eye_movement_types, eye_movement_ids, fixation_centroids = idt_algorithm(df, D_THRESHOLD, T_THRESHOLD)
df['Eye Movement ID'] = eye_movement_ids
df['Eye movement type - IDT'] = eye_movement_types

# Convert fixation centroids into a DataFrame and merge with the original data.
fixation_centroids_df = pd.DataFrame(fixation_centroids, columns=['Eye Movement ID', 'Fixation Centroid X', 'Fixation Centroid Y'])
df = df.merge(fixation_centroids_df, on='Eye Movement ID', how='left')


# Prints the few rows of the updated DataFrame to verify the results.
print(df.iloc[390:420])

     Unnamed: 0  Computer timestamp  Project name Participant name  \
390         487          3090207226  EyeTracking1     Participant1   
391         488          3091023903  EyeTracking1     Participant1   
392         489          3091040569  EyeTracking1     Participant1   
393         490          3091057236  EyeTracking1     Participant1   
394         491          3091690578  EyeTracking1     Participant1   
395         492          3091840580  EyeTracking1     Participant1   
396         493          3091857247  EyeTracking1     Participant1   
397         494          3091873914  EyeTracking1     Participant1   
398         495          3092440588  EyeTracking1     Participant1   
399         496          3092457255  EyeTracking1     Participant1   
400         497          3093290599  EyeTracking1     Participant1   
401         498          3093307266  EyeTracking1     Participant1   
402         499          3093773938  EyeTracking1     Participant1   
403         500     

As a result of this section, the main DataFrame now includes fixation points calculated using the I-DT algorithm. This dataframe is extracted from Tobii Pro Lab, it also has its classifications of fixations and saccades. You can compare between the implemented fixation and saccade classification and the classifications provided by Tobii Pro Lab. Note: The results may differ from those produced by Tobii Pro Lab due to variations in threshold settings and other filtering techniques. To align the outputs more closely, consider implementing additional filtering methods according to your specific needs.